# 順伝搬の実装

In [1]:
#パッケージのインポート
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms
import cv2 
import torch.nn.init as init

In [ ]:
#オフセット情報を使い、DBoxをBBoxに変換する関数
def decode(loc,dbox_list):
    """ 
    オフセット情報を使い、DBoxをBBoxに変換する。

    Parameteres
    ------------
    loc:[8732,4]
       SSDモデルで推論するオフセット情報
    dbox_list:[8732,4]
       DBoxの情報

    Returns
    ------------
    boxes:[xmin,ymin,xmax,ymax]
       BBoxの情報
    """

    #DBoxは[cx,cy,width,height]で格納されている
    #locも[△cx,△cy,△width,△height]で格納されている

    #オフセット情報からBBoxを求める
    boxes=torch.cat((
        dbox_list[:,:2]+loc[:,:2]*0.1*dbox_list[:,2:],
        dbox_list[:,:2]*torch.exp(loc[:,2:]*0.2)),dim=1)
    #boxesのサイズはtorch.Size([8732,4])となる

    #BBoxの座標情報を[cx,cy,width,height]から[xmin,ymin,xmax,ymax]に
    boxes[:,:2]-=boxes[:,2:]/2 #座標(xmin,ymin)へ変換
    boxes[:,2:]+=boxes[:,:2] #座標(xmax,ymax)へ変換

落書き開始

In [5]:
input1 = torch.randn(2, 3, 4)
input2 = torch.randn(2, 3, 4)

input_list = [input1, input2]
input_list

[tensor([[[-1.9924, -1.4571,  1.9735, -1.7305],
          [-1.4095,  0.4831,  0.6141, -1.0908],
          [ 1.3705, -0.4801,  0.0690,  0.8157]],
 
         [[-0.7145,  0.3737, -1.5077,  0.4178],
          [ 1.1349, -0.7082, -1.7310,  1.1911],
          [-1.4792, -0.5068,  1.5778,  0.7412]]]),
 tensor([[[-0.6165, -0.0142,  1.8006,  0.1723],
          [-0.1902, -0.2429,  0.3403, -0.3164],
          [ 0.1662, -0.0253, -0.0240,  0.3469]],
 
         [[-0.8588, -0.2077, -0.9160, -0.4949],
          [ 0.8555,  0.2658, -1.0679, -1.2622],
          [ 0.0213,  1.2953,  0.6868,  0.3789]]])]

In [6]:
torch.cat((input1,input2),dim=1)


tensor([[[-1.9924, -1.4571,  1.9735, -1.7305],
         [-1.4095,  0.4831,  0.6141, -1.0908],
         [ 1.3705, -0.4801,  0.0690,  0.8157],
         [-0.6165, -0.0142,  1.8006,  0.1723],
         [-0.1902, -0.2429,  0.3403, -0.3164],
         [ 0.1662, -0.0253, -0.0240,  0.3469]],

        [[-0.7145,  0.3737, -1.5077,  0.4178],
         [ 1.1349, -0.7082, -1.7310,  1.1911],
         [-1.4792, -0.5068,  1.5778,  0.7412],
         [-0.8588, -0.2077, -0.9160, -0.4949],
         [ 0.8555,  0.2658, -1.0679, -1.2622],
         [ 0.0213,  1.2953,  0.6868,  0.3789]]])

In [7]:
input1 = torch.randn(2, 3)
input2 = torch.randn(2, 3)

input_list = [input1, input2]
input_list

[tensor([[ 2.0333,  1.4100,  0.7338],
         [ 1.0435, -0.5074,  1.5584]]),
 tensor([[-1.3124, -0.0292, -0.3548],
         [-0.8827, -0.9851, -1.6132]])]

In [8]:
torch.cat((input1,input2),dim=1)


tensor([[ 2.0333,  1.4100,  0.7338, -1.3124, -0.0292, -0.3548],
        [ 1.0435, -0.5074,  1.5584, -0.8827, -0.9851, -1.6132]])

落書き終わり

## Non-Maximum Suppressionを行う関数を実装